# 구매 가망 고객 추출
    1. 목적 : 미래에 구매할 고객을 추출하여, 구매 행동이 존재하는 고객사들에 한하여 해당 고객에게 마케팅 액션을 취하여 KPI 증대에 기여
    2. 구성 : 전체적인 구성은 기존의 이탈 분석을 벤치마킹하여 구현
        2-1. 데이터 불러오기
            - 구매 행동(ex) event : SapBuy-)이 존재하는 고객사(appkey)에 대하여 진행 가능
            - X에 해당하는 독립변수의 기간은 28일, Y에 해당하는 종속변수의 기간은 7일로 설정
            - 학습 데이터와 예측 데이터는 7일의 기간 차이를 나타냄

        2-1. 데이터 전처리
            - basic_preprocess 함수 활용 : duration 전처리 
            - set_date_range 함수 활용 : 학습/예측 데이터 분리, label 생성
            - feature_engineering_pipe 함수 활용 : aggregation을 이용하여 최종 데이터셋 생성
          
        2-2. 데이터 샘플링
            - sampling_dataset 함수 활용 : 학습 데이터의 언더/오버 샘플링 진행
            
        2-3. 모델 학습
            - xgboost 모델 활용
            - 파라미터 튜닝은 미진행

        2-3. 모델 결과
            - test 데이터에 대한 예측 결과와 실제 결과 비교

    3. 결과 : 
        3-1. 자세한 검증 결과는 매출 예측 프로젝트 ppt에 존재함.
        3-2. 타 머신러닝 모델과의 성능 차이는 크게 나지 않지만, 해당 모델이 가장 성능이 높았음.
        
    4. 이슈 :
        4-1. 구매 관련한 변수를 추가할 수 있으나, 해당 기간(28일) 내에 구매하지 않은 고객들이 많아서 28일 설정에 대한 고민을 해보아야 함.
        4-2. label의 기간 설정이 7일로 되어 있어 그 이상으로 늘리는 시도를 해보아야 함.


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import time
import xgboost as xgb
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import sys
sys.path.append("/home/das_share/common_class/")
sys.path.append("/home/das_share/analysis/")
import DataImportClass
from DataImportClass import DataImport
import itertools

### 활용한 사용자정의 함수 리스트
    1. read_files(데이터 불러오기 함수) - 기존 이탈 분석 함수와 동일
    2. feature_engineering_pipe(변수 생성 함수) - 기존 이탈 분석 함수에서 변형
        - event 변수 생성 : 기간 내 평균 클릭 횟수
        - duration 변수 생성 : 기간 내 평균/최소/최대/표준편차 클릭 횟수
        - 접속 변수 생성 : 기존 이탈 분석에 사용된 변수들과 동일
        - 구매 접속 변수 생성 : 지난 7일 내 구매 횟수, 지난 4주 내 구매한 주
    3. make_events - events 컬럼에서 event name을 가져오는 함수
        - event 변수 생성 시 활용됨
    4. get_event_click - event의 횟수를 세는 함수
        - event 변수 생성 시 활용됨
    5. get_date_of_purchase - events 컬럼에서 event name 중 구매 이벤트의 날짜를 가져오는 함수
        - 구매 접속 변수 생성 시 활용됨
    6. get_sess_date - 지난 7일 내 접속한 날짜를 구하기 위한 함수
        - 접속 변수 생성 시 활용됨
    7. calculate_contis - 기존 이탈 분석 함수와 동일
        - 접속 변수 생성 시 활용됨
    8. sampling_dataset - 기존 이탈 분석 함수와 동일
        - 훈련 데이터의 오버샘플링 시 활용됨         

In [ ]:
def read_files(today, input_path, dates, platform_total = False):
    input_path_list = [input_path]
    if platform_total == True :
        input_path_list = []
        input_path_list.append(input_path.split('/')[0] +'/'+input_path.split('/')[1]+'/'+input_path.split('/')[2]+'/android')
        input_path_list.append(input_path.split('/')[0] +'/'+input_path.split('/')[1]+'/'+input_path.split('/')[2]+'/ios')  
    df_list = []
    for input_path in input_path_list :
        for i in range(1, dates+1):
            try : 
                date = today - timedelta(days=i) # 설정한 today를 기준으로 과거 n일 json
                y = str(date.year)[2:]
                m = str(date.month).zfill(2)
                d = str(date.day).zfill(2)
                filename = y+m+d+'.json'
                df_tmp = pd.read_json(input_path+'/'+filename)
                df_list.append(df_tmp)
            except ValueError :
                print("Not enough data to load.")
    df = pd.concat(df_list).reset_index(drop = True) # data merge
    return df

# 입력 데이터에 대하여 aggregation을 통한 변수를 만들어내는 함수
# 접속변수는 이탈분석에서 사용된 방법과 동일
def feature_engineering_pipe(df_tmp, churn, date1, date2, today):
    # aggregate event
    # sphereId 별 이벤트 클릭 수 저장
    df_tmp['events_name_list'] = df_tmp.events.apply(lambda x: make_events(x))
    df_event_gp = df_tmp.groupby('sphereId')['events_name_list'].apply(lambda x: list(itertools.chain(*x)))
    lst_event_unique = list(set(itertools.chain(*df_event_gp.values)))
    
    lst_events_gp = []
    for event in lst_event_unique:
        lst_events_gp.append(df_event_gp.apply(lambda x: get_event_click(x, event)).values)

    # aggregate duration
    # sphereId 별 접속시간 [평균, 합계, 표준편차, 최소값, 최대값] 저장
    lst_event_unique.append('duration__mean')
    lst_events_gp.append(df_tmp.groupby('sphereId')['duration'].mean().values)
    lst_event_unique.append('duration__sum')
    lst_events_gp.append(df_tmp.groupby('sphereId')['duration'].sum().values)   
    lst_event_unique.append('duration__std')
    lst_events_gp.append(df_tmp.groupby('sphereId')['duration'].std().values)
    lst_event_unique.append('duration__min')
    lst_events_gp.append(df_tmp.groupby('sphereId')['duration'].min().values)
    lst_event_unique.append('duration__max')
    lst_events_gp.append(df_tmp.groupby('sphereId')['duration'].max().values)
    # make data - event / duration
    df_events_gp = pd.DataFrame(lst_events_gp).T
    df_events_gp.columns = lst_event_unique
    df_events_gp['sphereId'] = df_event_gp.index.values
    df_events_gp = df_events_gp.fillna(0)
    
    # aggregate session
    # 접속 변수 생성 - 이탈분석에 사용되는 방법과 동일
    lst_sess_columns = ['sphereId','conti_act_days_last7','conti_act_weeks_last4','act_days_last7','act_weeks_last4',
                     'conti_inact_days_last7','conti_inact_weeks_last4','days_since_first','days_since_last']
    df_sess_gp = df_tmp.groupby('sphereId')['date'].apply(list).reset_index()
    e = today.isocalendar()[1]
    
    df_sess_gp['last7'] = df_sess_gp['date'].apply(lambda x: get_sess_date(x, date2))
    df_sess_gp['last4'] = df_sess_gp['date'].apply(lambda x: list(set(i.week for i in x)))
    df_sess_gp['inact_days'] = df_sess_gp['last7'].apply(lambda x: list(set([y for y in range(date2.day, date2.day+7)]) - set(x)))
    df_sess_gp['inact_weeks'] = df_sess_gp['last4'].apply(lambda x: list(set([y for y in range(e-3,e+1)]) - set(x)))
    
    df_sess_gp['conti_act_days_last7'] = df_sess_gp['last7'].apply(lambda x: calculate_contis(x))
    df_sess_gp['conti_act_weeks_last4'] = df_sess_gp['last4'].apply(lambda x: calculate_contis(x))
    df_sess_gp['conti_inact_days_last7'] = df_sess_gp['inact_days'].apply(lambda x: calculate_contis(x))
    df_sess_gp['conti_inact_weeks_last4'] = df_sess_gp['inact_weeks'].apply(lambda x: calculate_contis(x))
    df_sess_gp['act_days_last7'] = df_sess_gp['last7'].str.len()
    df_sess_gp['act_weeks_last4'] = df_sess_gp['last4'].str.len()
    df_sess_gp['days_since_first'] = df_sess_gp['date'].apply(lambda x: (today-x[0]).days)
    df_sess_gp['days_since_last'] = df_sess_gp['date'].apply(lambda x: (today-x[-1]).days)
    
    # make data - session
    df_sess_gp = df_sess_gp[lst_sess_columns]

    # aggregate purchase session date
    # 접속변수들과 동일하게 구매를 기준으로 하여 생성하려고 했으나, 대부분의 사람들이 구매를 하지 않기 때문에 오류가 발생하여
    # 지난 7일간 구매 횟수, 지난 4주간 구매 횟수 변수만 추가하였음
    df_tmp['date'] = pd.to_datetime(df_tmp['date'])
    df_tmp['date_purchase'] = df_tmp[['date','events']].apply(lambda x: get_date_of_purchase(x), axis=1)
    
    df_sess_purchase_gp = df_tmp.groupby('sphereId')['date_purchase'].apply(lambda x: list(itertools.chain(*x))).reset_index()
    df_sess_purchase_gp.columns = ['sphereId','date_purchase']
    lst_sess_purchase_columns = [i+"_purchase" for i in lst_sess_columns if i != "sphereId"]
    
    df_sess_purchase_gp['last7'] = df_sess_purchase_gp['date_purchase'].apply(lambda x: get_sess_date(x, date2))
    df_sess_purchase_gp['last4'] = df_sess_purchase_gp['date_purchase'].apply(lambda x: list(set(i.week for i in x)))

    df_sess_purchase_gp['act_days_last7'] = df_sess_purchase_gp['last7'].str.len()
    df_sess_purchase_gp['act_weeks_last4'] = df_sess_purchase_gp['last4'].str.len()

    # make data - purchase session date
    df_sess_purchase_gp = df_sess_purchase_gp[["sphereId","act_days_last7","act_weeks_last4"]]
    
    # make label
    # 구매 여부 변수 생성
    df_sess_gp['churn'] = df_sess_gp['sphereId'].map(lambda x: 1 if x in churn else 0)

    return df_events_gp, df_sess_gp, df_sess_purchase_gp

# 각 event들의 횟수를 구하기 위해 사용하는 함수
def make_events(x):
    lst_events = []
    for i in x:
        lst_events.append(i['name'])
    return lst_events

# 기간 내에 모든 구매 날짜를 저장하기 위해 사용하는 함수
def get_date_of_purchase(x):
    lst_dates = []
    if 'sapBuyStore' in str(x['events']):
        lst_dates.append(x['date'])
    return lst_dates

# 총 event 횟수를 구하기 위해 사용하는 함수
def get_event_click(x, event):
    return x.count(event)

# 지난 7일간 접속한 날짜를 구하기 위해 사용하는 함수
def get_sess_date(lst_sess, flag_date):
    result = []
    for sess in lst_sess:
        if sess > flag_date:
            result.append(sess)
        else:
            pass
        return list(set(result))

def calculate_contis(lst): 
    output = 0
    if len(lst) == 0:
        return 0
    elif len(lst) == 1:
        return 1
    else:
        conti = 1
        for i in range(len(lst)-1):
            if lst[i+1] - lst[i] == 1:
                conti += 1
                if conti > output:
                    output = conti
            else:
                conti = 1
        return output
    
# train 데이터를 샘플링하는 함수
def sampling_dataset(X_train, y_train):
    #smote = SMOTE(n_jobs=6, random_state=42)
    #smote = RandomOverSampler(random_state=42)
    undersample = RandomUnderSampler(sampling_strategy='majority')
    cols = X_train.columns
    X_train, y_train = undersample.fit_resample(X_train, y_train)
    X_train = pd.DataFrame(X_train, columns=cols)
    X_train = X_train.fillna(0)
    y_train = pd.Series(y_train)
    return X_train, y_train

Using TensorFlow backend.


### 데이터 불러오기 및 조건 설정
    - 한번에 여러번의 분석 일자를 테스팅하기 위해, 기간을 길게 설정 후 데이터를 불러옴

In [ ]:
# 계속 데이터를 불러오지 않고, 한번에 긴 데이터를 불러와서 저장하고 이를 사용했습니다.
# 1) data import
today = datetime(2020,5,31)
input_path = '/home/heemok/tand/data_coke/coke_android'#'../in/coke/android'
key_id = 'sphereId'
dates = 50
df = read_files(today, input_path, dates, platform_total)

# 2) Basic preprocess
df = basic_preprocess(df)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d %H:%M:%S').dt.date

### 분석 실행
    - for문을 활용하여, 앞서 불러온 전체 데이터를 분석일자에 맞게만 사용하면서 분석을 진행함
    - 분석일자별로 리스트에 성능결과가 저장됨

In [ ]:
result = []
# 날짜별 분석 실행하여 분석 결과를 저장
for today in [datetime(2021,1,31) - timedelta(days=i) for i in range(30)]:
    print(today)
    start_time = time.time()

    date1 = today - timedelta(days=35)
    date2 = today - timedelta(days=28)
    date3 = today - timedelta(days=14)
    date4 = today - timedelta(days=7)
    date5 = today + timedelta(days=7)

    # Set Windows
    train_window = DataImport.set_date_range(date1, date4, df)
    train_label = DataImport.set_date_range(date4, today, df)

    test_window = DataImport.set_date_range(date2, today, df)
    test_label = DataImport.set_date_range(today, date5, df)

    # create purchase_label
    label_df = train_label.groupby('sphereId')['events'].apply(list).reset_index()
    label_df['events'] = label_df.events.astype('str').apply(lambda x: 1 if 'sapBuyStore' in x else 0)
    label_df_test = test_label.groupby('sphereId')['events'].apply(list).reset_index()
    label_df_test['events'] = label_df_test.events.astype('str').apply(lambda x: 1 if 'sapBuyStore' in x else 0)

    # set label
    train_ids = set(train_window['sphereId'])
    test_ids = set(test_window['sphereId'])

    train_label_ids = set(label_df.loc[label_df['events']==1].sphereId)
    train_churn = train_label_ids#train_ids - train_label_ids
    test_label_ids = set(label_df_test.loc[label_df_test['events']==1].sphereId)
    test_churn = test_label_ids#test_ids - test_label_ids
    
    train_window['date'] = pd.to_datetime(train_window['date'])
    test_window['date'] = pd.to_datetime(test_window['date'])
    
    # feature engineering
    df_events_gp_train, df_sess_gp_train, df_sess_purchase_gp_train = feature_engineering_pipe(train_window, train_churn, date1, date3, date4)
    df_events_gp_test, df_sess_gp_test, df_sess_purchase_gp_test = feature_engineering_pipe(test_window, test_churn, date2, date4, today)
    
    # merge data
    df_train = pd.merge(df_events_gp_train, df_sess_gp_train, on='sphereId', how='outer')
    df_train = pd.merge(df_train, df_sess_purchase_gp_train, on='sphereId', how='outer')
    df_test = pd.merge(df_events_gp_test, df_sess_gp_test, on='sphereId', how='outer')
    df_test = pd.merge(df_test, df_sess_purchase_gp_test, on='sphereId', how='outer')
    
    # prepare data
    X_train, X_test = df_train.drop(['sphereId','churn'], axis=1), df_test.drop(['sphereId','churn'], axis=1)
    y_train, y_test = df_train['churn'], df_test['churn']
    
    # sampling train data
    X_train, y_train = sampling_dataset(X_train, y_train)

    feat = list(set(X_test.columns)&set(X_train.columns))
    X_train = X_train[feat]
    X_test = X_test[feat].fillna(0)
    
    # modelling
    model = xgb.XGBClassifier(objective='binary:logistic', random_state=42, n_jobs=-1)
    model.fit(X_train.values,y_train.values)

    y_pred = model.predict(X_test.values)
    y_pred_proba = model.predict_proba(X_test.values)
    
    # evaluation
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
    result.append([precision,recall,f1,auc])
    
    print(confusion_matrix(y_test, y_pred))
    print('precision : ',precision)
    print('recall : ', recall)
    print('f1 : ', f1)
    print('auc : ', auc)
    print("--- {}s seconds---".format(time.time()-start_time))

2021-01-31 00:00:00


/home/heemok/.local/share/virtualenvs/heemok-SnMGMROh/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:31:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[13017  5404]
 [   39   330]]
precision :  0.05755144750610394
recall :  0.8943089430894309
f1 :  0.1081435359659184
auc :  0.8771766757893409
--- 37.40783214569092s seconds---
2021-01-30 00:00:00


KeyboardInterrupt: 